# PROJECT KAYAK

In [1]:
## Import of the necessary libraries

import requests
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
import numpy as np

import os
import logging
import pandas as pd
!pip install scrapy
import scrapy
from scrapy.crawler import CrawlerProcess

import plotly.express as px
import plotly.io as pio

!pip install boto3
!pip install psycopg2-binary
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker 



In [2]:
# Definition of the 35 selected cities for the project

cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

## PART 1 : Data collection
### A - GPS Coordinates

In [3]:
# On va d'abord recuperer les latitudes et longitudes, dans l'ordre, de nos villes concernées

cities_bis = [c.replace(' ','+') for c in cities]
gpslat = []
gpslon = []

for city in cities_bis:
    r = requests.get('https://nominatim.openstreetmap.org/search?q={}&country=france&format=json'.format(city))
    gpslat.append(r.json()[0]['lat'])
    gpslon.append(r.json()[0]['lon'])
 

In [4]:
print(gpslat)

['48.6359541', '48.649518', '49.2764624', '49.4938975', '49.4404591', '48.8588897', '49.8941708', '50.6365654', '48.584614', '48.249489800000006', '48.0777517', '48.0447968', '47.2380222', '47.3215806', '45.8992348', '45.1875602', '45.7578137', '43.7496562', '43.1506968', '43.2140359', '43.2961743', '43.5298424', '43.9492493', '44.0121279', '43.8374249', '43.5658225', '43.4522771', '42.52505', '43.2130358', '42.9455368', '43.6044622', '44.0175835', '43.471143749999996', '43.4945144', '46.1591126']


In [5]:
print(gpslon)

['-1.511459954959514', '-2.0260409', '-0.7024738', '0.1079732', '1.0939658', '2.3200410217200766', '2.2956951', '3.0635282', '7.7507127', '7.34429620253195', '7.3579641', '7.3079618', '6.0243622', '5.0414701', '6.1288847', '5.7357819', '4.8320114', '6.3285616', '6.3419285', '5.5396318', '5.3699525', '5.4474738', '4.8059012', '4.4196718', '4.3600687', '4.1912837', '4.4287172', '3.0831554', '2.3491069', '1.4065544156065486', '1.4442469', '1.3549991', '-1.552726590666314', '-1.4736657', '-1.1520434']


In [6]:
# Assembling collecting gps coordinates

list_df_gps = [gpslat,gpslon]
df_gps = pd.DataFrame(list_df_gps,columns = cities, index = ('City_Lat', 'City_Lon'))
df_gps = df_gps.transpose()
df_gps['City'] = df_gps.index

In [7]:
display(df_gps)

,City_Lat,City_Lon,City
Mont Saint Michel,48.6359541,-1.511459954959514,Mont Saint Michel
St Malo,48.649518,-2.0260409,St Malo
Bayeux,49.2764624,-0.7024738,Bayeux
Le Havre,49.4938975,0.1079732,Le Havre
Rouen,49.4404591,1.0939658,Rouen
Paris,48.8588897,2.3200410217200766,Paris
Amiens,49.8941708,2.2956951,Amiens
Lille,50.6365654,3.0635282,Lille
Strasbourg,48.584614,7.7507127,Strasbourg
Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195,Chateau du Haut Koenigsbourg


### B - Weather informations

In [8]:
# Requesting weather data from API

t_pop = []
t_temp = []
t_weather = []


for i in range (35):
    t = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units=metric&appid=37fd61377390482a5e5f103a29520af8'.format(gpslat[i],gpslon[i]))
    for day in range(1,8):
        t_pop.append(t.json()['daily'][day]['pop'])
        t_temp.append(t.json()['daily'][day]['temp'])
        t_weather.append(t.json()['daily'][day]['weather'])



In [9]:
# quick check
t_weather[0]

[{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}]

In [10]:
# Extracting usefull data from raw informations

weather_descript_temp = []
temp_of_day_temp = []

for x in range(245):  # extracting weather descriptions
    weather_descript_temp.append(t_weather[x][0]['description'])

temp_of_day_temp.append([d['day'] for d in t_temp]) # extracting day temperatue


In [11]:
# We have all our data but they are not sorted. We will create a list of lists to sort them by city for 7 days

weather_descript = []
precipitation = []
temp_of_day_temp = temp_of_day_temp[0]
temp_of_day = []

weather_descript = [weather_descript_temp[x:x+7] for x in range(0, len(weather_descript_temp),7)]
temp_of_day = [temp_of_day_temp[x:x+7] for x in range(0, len(temp_of_day_temp),7)]
precipitation = [t_pop[x:x+7] for x in range(0, len(t_pop),7)]

In [12]:
# Assembling all the weather informations in one dataframe

df_weather = pd.DataFrame()
df_weather['City'] = cities
df_weather['7days_temperature'] = temp_of_day
df_weather['7days_precipitation'] = precipitation
df_weather['7days_weather_description'] = weather_descript

df_weather

,City,7days_temperature,7days_precipitation,7days_weather_description
0,Mont Saint Michel,"[24.73, 21.91, 21.6, 20.33, 22.72, 26.57, 29.92]","[0.73, 0.45, 0.06, 0.62, 0.88, 0, 0.94]","[light rain, light rain, scattered clouds, lig..."
1,St Malo,"[21.91, 20.34, 20.25, 20.06, 20.8, 23.49, 24.99]","[0.81, 0.53, 0, 0.84, 0.76, 0, 0.87]","[light rain, light rain, scattered clouds, lig..."
2,Bayeux,"[22.71, 20.9, 22.44, 18.33, 22.53, 26.45, 31.56]","[0.62, 0.69, 0, 0.35, 1, 0, 0.9]","[light rain, light rain, clear sky, light rain..."
3,Le Havre,"[22.14, 20.64, 20.21, 19.07, 21.04, 22.34, 28.55]","[0.7, 0.76, 0, 0.3, 1, 0, 0.53]","[light rain, light rain, clear sky, light rain..."
4,Rouen,"[27.6, 23.92, 24.49, 19.88, 23.55, 26.76, 31.37]","[0.51, 0.64, 0, 0.08, 1, 0, 0.05]","[clear sky, light rain, clear sky, overcast cl..."
5,Paris,"[27.92, 25.71, 25.84, 23.48, 21.46, 27.96, 31.64]","[0.46, 0.28, 0.09, 0, 0.95, 0.25, 0]","[scattered clouds, few clouds, few clouds, ove..."
6,Amiens,"[26.58, 24.31, 24.1, 24.99, 24.12, 25.4, 30.35]","[0.82, 0.84, 0, 0.03, 1, 0, 0]","[light rain, moderate rain, clear sky, overcas..."
7,Lille,"[26.07, 25.44, 24.46, 23.97, 23.61, 25.74, 30.05]","[0.93, 1, 0.02, 0.02, 1, 0.14, 0]","[light rain, moderate rain, clear sky, overcas..."
8,Strasbourg,"[23.72, 26.41, 27.52, 27.45, 17.3, 25.8, 30.69]","[1, 0.27, 0.32, 0, 0.68, 0.08, 0]","[moderate rain, light rain, light rain, broken..."
9,Chateau du Haut Koenigsbourg,"[18.94, 23.29, 24.11, 24.56, 13.94, 24.05, 26.85]","[1, 0.32, 0.4, 0, 0.62, 0.15, 0]","[moderate rain, light rain, light rain, broken..."


### C - Hotels informations 


In [13]:
# Define the spider to crawl pages and get the data needed
class bookingSpider(scrapy.Spider):
    # Define variables
    locs = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigue sMortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]
    name = "booking"
    init_url = dict() # set a dictionary for the urls to follow
    start_urls = ["https://www.booking.com/"]

    # Method to enter the cities in searching destination
    def parse(self, response):
        for loc in self.locs:
            # returns a new FormRequest object with its form field values pre-populated with those found in the HTML <form> element contained in the given response.
            yield scrapy.FormRequest.from_response(response,
                                                   formdata = {'ss': loc},
                                                   callback = self.after_search,
                                                   cb_kwargs = {'location':loc, 'page_no':0}
                                                   )

    # Method to find name, url, score and description of the hotels
    def after_search(self, response, location, page_no):
        # Condition to crawl pages
        if page_no == 0:
            self.init_url[location] = response.url
        
        cities = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for city in cities:
            name = city.css('div.fcab3ed991.a23c043802::text').get()
            url = city.css('a.e13098a59f').attrib['href']
            score = city.css('div.b5cd09854e.d10a6220b4::text').get()
            desc = city.css('div.d8eab2cf7f::text').get()
            
            dic = {'location': location,
                    'name': name,
                    'url': url,
                    'score':score,
                    'description': desc
                  }
            # initiate method for the parse_hotel GPS cordonnates 
            try: 
                yield response.follow(url = url,
                                      callback = self.GPS, 
                                      cb_kwargs = {'dic': dic}
                                     ) 
            except:
                dic['lat'] = None
                dic['lon'] = None
                yield dic
         
        if page_no <= 1: 
            # Create variable next_page by the url and end string to give to follow function below
            next_page = self.init_url[location]+"&offset="+str(25*(page_no+1)) 
            # follow method return a request instance to follow a link, here it is the next_page variable who contain previous url with a page tag at the end
            yield response.follow(next_page,
                                  callback = self.after_search,
                                  cb_kwargs = {'location':location, 'page_no':page_no+1}
                                 ) 

    # method to find GPS coordonates and split them in two variables lat & Lon        
    def GPS(self, response, dic): 
        try: 
            gps = response.css('a.jq_tooltip.loc_block_link_underline_fix.bui-link.show_on_map_hp_link.show_map_hp_link').attrib['data-atlas-latlng']   
        except:
            dic['lat']=None
            dic['lon']=None
            yield dic
            return None
        try:
            gps = gps.split(',')
            dic['lat']=float(gps[0])
            dic['lon']=float(gps[1])
            yield dic
        except:
            print("No way to split")
            dic['lat']=None
            dic['lon']=None
            yield dic
            
# Define a filename
filename = "scrap_booking.json"

# Remove previous file if it allready exist
if filename in os.listdir('hotels/'):
        os.remove('hotels/' + filename)

# Define the settings for the crawler process
process = CrawlerProcess(settings = {
    # simulate browser
    'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0', 
    # display regular errors
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'hotels/' + filename : {"format": "json"},
    }
})

# Initiate crawler
process.crawl(bookingSpider)
process.start()

In [14]:
df_hotels = pd.read_json('hotels/scrap_booking.json')
df_hotels.shape

(2608, 7)

### D - Cleaning and organising data

In [15]:
# Checking if there some missing informations and removing the according line if so

for column in df_hotels:
    if df_hotels[column].isnull().values.any():
        df_hotels = df_hotels[df_hotels[column].isna() == False]
        
df_hotels.shape


(2520, 7)

In [16]:
# Rename location column to city
df_hotels.rename({'location': 'City'}, axis=1, inplace=True)

In [17]:
# Merging all out dataframes
df_complete = df_hotels.merge(df_weather , how = 'left' , on = 'City')
df_complete = df_complete.merge(df_gps , how = 'left' , on ='City')


In [18]:
# Correcting datatype
df_complete['City_Lat'] = pd.to_numeric(df_complete['City_Lat'])
df_complete['City_Lon'] = pd.to_numeric(df_complete['City_Lon'])

In [19]:
# Saving the data in a csv file
df_complete.to_csv('Complete_infos' , encoding='utf-8', index=False)

## PART 2 - Data Storage

In [22]:
# Using boto3 to create and connect a S3 bucket

session = boto3.Session(aws_access_key_id='XXXXXXXXXXXX', aws_secret_access_key='XXXXXXXXXXXXXXXX')
s3 = session.resource('s3')

bucket = s3.create_bucket(Bucket='project-kayak-bucket2' , CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [23]:
# Uploading the csv file into the bucket

bucket.put_object(Key ='Complete_infos' , Body= 'Complete_infos.csv')

s3.Object(bucket_name='project-kayak-bucket2', key='Complete_infos')

In [24]:
# Connecting to the sql database

DBHOST = 'twix-kayak-database.cxqcbdx5rdug.eu-west-3.rds.amazonaws.com'
DBUSER = 'XXXX'
DBPASS = 'XXXXXX'
DBNAME = 'postgres'

engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)


In [26]:
# Initialize a sessionmaker to exchange with our database

Session = sessionmaker(bind = engine)

# Instanciate session 

session = Session()

df_complete.to_sql('Kayak_complete_infos_df2', engine)

2022-08-17 17:43:54,726 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-08-17 17:43:54,727 INFO sqlalchemy.engine.Engine [cached since 10.6s ago] {'name': 'Kayak_complete_infos_df2'}
2022-08-17 17:43:54,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-17 17:43:54,752 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Kayak_complete_infos_df2" (
	index BIGINT, 
	"City" TEXT, 
	name TEXT, 
	url TEXT, 
	score FLOAT(53), 
	description TEXT, 
	lat FLOAT(53), 
	lon FLOAT(53), 
	"7days_temperature" TEXT, 
	"7days_precipitation" TEXT, 
	"7days_weather_description" TEXT, 
	"City_Lat" FLOAT(53), 
	"City_Lon" FLOAT(53)
)


2022-08-17 17:43:54,753 INFO sqlalchemy.engine.Engine [no key 0.00106s] {}
2022-08-17 17:43:54,773 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Kayak_complete_infos_df2_index" ON "Kayak_complete_infos_df2" (index)
2022-08-17 17:43:54,774 INFO sqla

# PART 3 - Temperature and Hotels visualisation

In [27]:
# Getting the average temperature and precipitation for each city

df_complete['Average_temperature'] = df_complete['7days_temperature'].apply(np.mean)
df_complete['Average_precipitation'] = df_complete['7days_precipitation'].apply(np.mean)

In [28]:
# Selecting the 5 best destinations for the week / to make my selection I choose to filter first on the 10 cities where it is going to rain the less then I picked the top 5 according to the temperature

top5cities = df_complete.groupby('City').mean().sort_values('Average_precipitation', ascending = True).head(10)
top5cities = top5cities.sort_values('Average_temperature', ascending= False).head()

In [29]:
display(top5cities)

,score,lat,lon,City_Lat,City_Lon,Average_temperature,Average_precipitation
City,,,,,,,
Nimes,8.449296,43.835905,4.358426,43.837425,4.360069,30.562857,0.144286
Uzes,8.810145,44.011582,4.418398,44.012128,4.419672,29.537143,0.165714
Avignon,8.596000,43.947031,4.814772,43.949249,4.805901,29.497143,0.208571
Carcassonne,8.629333,43.209500,2.358179,43.213036,2.349107,29.037143,0.168571
Collioure,8.462500,42.524909,3.084577,42.525050,3.083155,28.961429,0.162857


In [33]:
# Selecting the 20 best hotels according to the top 5 cities

hotels_shortlist = df_complete[ df_complete['City'].isin(top5cities.index)]
hotels_shortlist = hotels_shortlist.sort_values('score' , ascending= False).head(20)
display(hotels_shortlist)

,City,name,url,score,description,lat,lon,7days_temperature,7days_precipitation,7days_weather_description,City_Lat,City_Lon,Average_temperature,Average_precipitation
1692,Collioure,BEL APPARTEMENT 4Pers RDC/parking/terrasse/ ja...,https://www.booking.com/hotel/fr/bel-apparteme...,10.0,BEL APPARTEMENT 4Pers RDC/parking/terrasse/ ja...,42.528363,3.078163,"[26.71, 27.64, 30.07, 30.08, 29.85, 28.5, 29.88]","[0.72, 0, 0, 0, 0, 0, 0.42]","[light rain, clear sky, clear sky, broken clou...",42.525050,3.083155,28.961429,0.162857
1307,Avignon,Lumineuse Maison de Ville - Piscine - Intramuros,https://www.booking.com/hotel/fr/lumineuse-mai...,10.0,Managed by a private host,43.944238,4.811159,"[22.33, 27.95, 30.41, 29.89, 30.33, 31.61, 33.96]","[0.99, 0.47, 0, 0, 0, 0, 0]","[moderate rain, light rain, clear sky, scatter...",43.949249,4.805901,29.497143,0.208571
1637,Nimes,"Un petit PARADIS, CENTRE VILLE de Nîmes",https://www.booking.com/hotel/fr/un-petit-para...,10.0,Managed by a private host,43.836316,4.358485,"[24.75, 30.17, 31.88, 31.56, 31.6, 31.08, 32.9]","[0.87, 0.14, 0, 0, 0, 0, 0]","[light rain, clear sky, clear sky, broken clou...",43.837425,4.360069,30.562857,0.144286
1652,Nimes,La Pimprenelle - Enjoy the peace and quiet in ...,https://www.booking.com/hotel/fr/la-pimprenell...,10.0,"Situated in Nîmes, 7 km from Parc Expo Nîmes a...",43.879131,4.377070,"[24.75, 30.17, 31.88, 31.56, 31.6, 31.08, 32.9]","[0.87, 0.14, 0, 0, 0, 0, 0]","[light rain, clear sky, clear sky, broken clou...",43.837425,4.360069,30.562857,0.144286
2098,Uzes,la maison des Olivettes,https://www.booking.com/hotel/fr/la-maison-des...,10.0,Managed by a private host,43.998528,4.411169,"[23, 28.91, 30.83, 30.55, 30.57, 30.11, 32.79]","[0.96, 0.2, 0, 0, 0, 0, 0]","[moderate rain, clear sky, clear sky, broken c...",44.012128,4.419672,29.537143,0.165714
1357,Avignon,Le Clos Saluces,https://www.booking.com/hotel/fr/le-clos-saluc...,9.9,Offering a flowered furnished garden and a pat...,43.950193,4.810586,"[22.33, 27.95, 30.41, 29.89, 30.33, 31.61, 33.96]","[0.99, 0.47, 0, 0, 0, 0, 0]","[moderate rain, light rain, clear sky, scatter...",43.949249,4.805901,29.497143,0.208571
2079,Uzes,Le domaine des Oréades,https://www.booking.com/hotel/fr/le-domaine-de...,9.8,Managed by a private host,44.037437,4.452160,"[23, 28.91, 30.83, 30.55, 30.57, 30.11, 32.79]","[0.96, 0.2, 0, 0, 0, 0, 0]","[moderate rain, clear sky, clear sky, broken c...",44.012128,4.419672,29.537143,0.165714
2136,Nimes,Antichambre,https://www.booking.com/hotel/fr/antichambre.e...,9.8,Located within 1.9 km of Parc Expo Nîmes and a...,43.834472,4.356771,"[24.75, 30.17, 31.88, 31.56, 31.6, 31.08, 32.9]","[0.87, 0.14, 0, 0, 0, 0, 0]","[light rain, clear sky, clear sky, broken clou...",43.837425,4.360069,30.562857,0.144286
1628,Nimes,Le Chant Des Oiseaux,https://www.booking.com/hotel/fr/le-chant-des-...,9.8,Managed by a private host,43.859178,4.370178,"[24.75, 30.17, 31.88, 31.56, 31.6, 31.08, 32.9]","[0.87, 0.14, 0, 0, 0, 0, 0]","[light rain, clear sky, clear sky, broken clou...",43.837425,4.360069,30.562857,0.144286
2061,Uzes,Appartement de charme dans le cœur historique ...,https://www.booking.com/hotel/fr/appartement-d...,9.7,Managed by a private host,44.012505,4.420930,"[23, 28.91, 30.83, 30.55, 30.57, 30.11, 32.79]","[0.96, 0.2, 0, 0, 0, 0, 0]","[moderate rain, clear sky, clear sky, broken c...",44.012128,4.419672,29.537143,0.165714


In [39]:
# Display weather map
fig = px.scatter_mapbox(top5cities,
                        title='Weather map of top 5 cities',
                        lat = 'City_Lat',
                        lon = 'City_Lon',
                        color = 'Average_temperature',
                        color_continuous_scale = px.colors.diverging.balance,
                        size = 'Average_temperature',
                        size_max = 30,
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  template = 'plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2, "lat" : 47}, "zoom": 4.8},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )

fig.show()

In [42]:
# Display Top 20 hotels for the top 5 cities
fig = px.scatter_mapbox(hotels_shortlist,
                        title = '20 Best Hotels',
                        lat = 'lat',
                        lon = 'lon',
                        color = 'score',
                        size = 'score',
                        color_continuous_scale = px.colors.diverging.Tropic,
                        size_max = 20,
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  template='plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2.6, "lat" : 43.5}, "zoom": 7},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )
fig.show()